In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

In [19]:
NUM_SAMPLES = 20000    
MAX_SEQ_LEN = 5         
VOCAB_SIZE = 11         
EMBED_DIM = 128
LATENT_DIM = 256
BATCH_SIZE = 64
EPOCHS = 300

In [ ]:
def generate_data(num_samples=10000, max_len=7, vocab_size=11):
    X, Y = [], []
    for _ in range(num_samples):
        seq_len = np.random.randint(2, max_len + 1)
        seq = np.random.randint(1, vocab_size - 1, size=seq_len).tolist()
        X.append(seq)
        Y.append(seq[::-1])
    return X, Y

input_texts, target_texts = generate_data(NUM_SAMPLES, MAX_SEQ_LEN, VOCAB_SIZE)

In [21]:
START_TOKEN = VOCAB_SIZE - 2
END_TOKEN = VOCAB_SIZE - 1

encoder_input = tf.keras.preprocessing.sequence.pad_sequences(input_texts, maxlen=MAX_SEQ_LEN, padding='post')

decoder_input = [[START_TOKEN] + seq for seq in target_texts]
decoder_output = [seq + [END_TOKEN] for seq in target_texts]

decoder_input = tf.keras.preprocessing.sequence.pad_sequences(decoder_input, maxlen=MAX_SEQ_LEN+1, padding='post')
decoder_output = tf.keras.preprocessing.sequence.pad_sequences(decoder_output, maxlen=MAX_SEQ_LEN+1, padding='post')

In [22]:
encoder_inputs = layers.Input(shape=(MAX_SEQ_LEN,))
enc_emb = layers.Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(encoder_inputs)
encoder_lstm = layers.LSTM(LATENT_DIM, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

decoder_inputs = layers.Input(shape=(MAX_SEQ_LEN+1,))
dec_emb = layers.Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(decoder_inputs)
decoder_lstm = layers.LSTM(LATENT_DIM, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = layers.Dense(VOCAB_SIZE, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 5, 128)    │      1,408 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_5         │ (None, 5)         │          0 │ input_layer_8[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 6, 128)    │      1,408 │ input_layer_9[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 256),     │    394,240 │ embedding_5[0][0… │
│                     │ (None, 256),      │            │ not_equal_5[0][0] │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 6, 256),  │    394,240 │ embedding_6[0][0… │
│                     │ (None, 256),      │            │ lstm_4[0][1],     │
│                     │ (None, 256)]      │            │ lstm_4[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 6, 11)     │      2,827 │ lstm_5[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 794,123 (3.03 MB)

 Trainable params: 794,123 (3.03 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

history = model.fit(
    [encoder_input, decoder_input],
    np.expand_dims(decoder_output, -1),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
    callbacks=[early_stop]
)

Epoch 1/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 21s 57ms/step - accuracy: 0.5845 - loss: 0.6436 - val_accuracy: 0.7587 - val_loss: 0.0360
Epoch 2/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 16s 56ms/step - accuracy: 0.7477 - loss: 0.0163 - val_accuracy: 0.7601 - val_loss: 0.0068
Epoch 3/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 16s 55ms/step - accuracy: 0.7469 - loss: 0.0107 - val_accuracy: 0.7602 - val_loss: 0.0031
Epoch 4/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 17s 59ms/step - accuracy: 0.7483 - loss: 0.0015 - val_accuracy: 0.7602 - val_loss: 0.0011
Epoch 5/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 16s 56ms/step - accuracy: 0.7483 - loss: 8.0255e-04 - val_accuracy: 0.7602 - val_loss: 7.1163e-04
Epoch 6/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 17s 61ms/step - accuracy: 0.7483 - loss: 5.4062e-04 - val_accuracy: 0.7602 - val_loss: 5.2072e-04
Epoch 7/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 17s 61ms/step - accuracy: 0.7483 - loss: 3.9010e-04 - val_accuracy: 0.7602 - val_loss: 3.8880e-04
Epoch 8/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 17s 61ms/step - ac

In [29]:
encoder_model = models.Model(encoder_inputs, encoder_states)

decoder_state_input_h = layers.Input(shape=(LATENT_DIM,))
decoder_state_input_c = layers.Input(shape=(LATENT_DIM,))
dec_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding_layer = layers.Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)
dec_emb2 = decoder_embedding_layer(decoder_inputs)

decoder_lstm_layer = decoder_lstm
decoder_dense_layer = decoder_dense

dec_outputs2, state_h2, state_c2 = decoder_lstm_layer(dec_emb2, initial_state=dec_states_inputs)
dec_states2 = [state_h2, state_c2]
dec_outputs2 = decoder_dense_layer(dec_outputs2)

decoder_model = models.Model(
    [decoder_inputs] + dec_states_inputs,
    [dec_outputs2] + dec_states2
)

In [25]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = START_TOKEN
    stop_condition = False
    decoded = []

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == END_TOKEN or len(decoded) >= MAX_SEQ_LEN:
            stop_condition = True
        else:
            decoded.append(sampled_token_index)

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded


In [27]:
test_seq = np.array([ [1, 5, 7, 3, 0] ])
decoded = decode_sequence(test_seq)
print("Input :", test_seq[0])
print("Predicted reversed:", decoded)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Input : [1 5 7 3 0]
Predicted reversed: [np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(5)]
